## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

2025-03-05 21:24:38.055169: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]

In [4]:
# Create a list of at least 10 column names to use as X data
X_columns = ['Education',
            'Age',
            'DistanceFromHome',
            'JobSatisfaction',
            'OverTime', 'StockOptionLevel',
            'WorkLifeBalance',
            'YearsAtCompany',
            'YearsSinceLastPromotion',
            'NumCompaniesWorked']

# Create X_df using your selected columns
X_df = attrition_df[X_columns]

# Show the data types for X_df
X_df.dtypes

Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [5]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=1)

In [6]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train['OverTime'] = X_train['OverTime'].apply(lambda x: 1 if x == 'Yes' else 0)
X_test['OverTime'] = X_test['OverTime'].apply(lambda x: 1 if x == 'Yes' else 0)
X_train['OverTime'].value_counts()


OverTime
0    798
1    304
Name: count, dtype: int64

In [7]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_scaler = scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
department_enc = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit the encoder to the training data
department_enc.fit(np.array(y_train['Department']).reshape(-1, 1))

# Create two new variables by applying the encoder
# to the training and testing data
y_train_department_enc = department_enc.transform(np.array(y_train['Department']).reshape(-1, 1))
y_test__department_enc = department_enc.transform(np.array(y_test['Department']).reshape(-1, 1)) 
y_train_department_enc

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [9]:
# Create a OneHotEncoder for the Attrition column
attrition_enc = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit the encoder to the training data
attrition_enc.fit(np.array(y_train['Attrition']).reshape(-1, 1))

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition_enc = attrition_enc.transform(np.array(y_train['Attrition']).reshape(-1, 1)) 
y_test__attrition_enc = attrition_enc.transform(np.array(y_test['Attrition']).reshape(-1, 1))  
y_train_attrition_enc

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

## Part 2: Create, Compile, and Train the Model

In [10]:
# Find the number of columns in the X training data.
input_features = len(X_train.columns)

# Create the input layer
input = layers.Input(shape=(input_features,))

# Create at least two shared layers
shared_layer1 = layers.Dense(units=64, activation='relu', name='shared1')(input)
shared_layer2 = layers.Dense(units=32, activation='relu', name='shared2')(shared_layer1)

2025-03-05 21:24:43.296105: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = layers.Dense(32, activation='relu',
                                 name='department_hidden')(shared_layer2)

# Create the output layer
department_output = layers.Dense(y_train_department_enc.shape[1],
                                 activation='softmax',
                                 name='department_output')(department_hidden)

In [12]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden= layers.Dense(32, activation='relu',
                                 name='attrition_hidden')(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(y_train_attrition_enc.shape[1], activation='sigmoid', name='attrition_output')(attrition_hidden)


In [13]:
# Create the model
model = Model(inputs=input, outputs={'department_output': department_output, 'attrition_output': attrition_output}) 

# Compile the model
model.compile(optimizer='adam', loss={ 
            'department_output': 'categorical_crossentropy',
            'attrition_output': 'binary_crossentropy' }, 
            metrics={ 'department_output': 'accuracy',
                     'attrition_output': 'accuracy' })

# Summarize the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 10)]         0           []                               
                                                                                                  
 shared1 (Dense)                (None, 64)           704         ['input_1[0][0]']                
                                                                                                  
 shared2 (Dense)                (None, 32)           2080        ['shared1[0][0]']                
                                                                                                  
 attrition_hidden (Dense)       (None, 32)           1056        ['shared2[0][0]']                
                                                                                              

In [14]:
# Train the model
model.fit(X_train_scaled, {'department_output': y_train_department_enc, 'attrition_output': y_train_attrition_enc},
                    epochs=100, validation_split=0.2)

Epoch 1/100
28/28 [==============================] - 1s 10ms/step - loss: 1.6685 - attrition_output_loss: 0.6460 - department_output_loss: 1.0225 - attrition_output_accuracy: 0.7037 - department_output_accuracy: 0.4938 - val_loss: 1.4953 - val_attrition_output_loss: 0.5704 - val_department_output_loss: 0.9249 - val_attrition_output_accuracy: 0.8281 - val_department_output_accuracy: 0.6335
Epoch 2/100
28/28 [==============================] - 0s 2ms/step - loss: 1.3074 - attrition_output_loss: 0.4883 - department_output_loss: 0.8192 - attrition_output_accuracy: 0.8513 - department_output_accuracy: 0.6640 - val_loss: 1.3455 - val_attrition_output_loss: 0.4899 - val_department_output_loss: 0.8556 - val_attrition_output_accuracy: 0.8281 - val_department_output_accuracy: 0.6335
Epoch 3/100
28/28 [==============================] - 0s 2ms/step - loss: 1.1892 - attrition_output_loss: 0.4154 - department_output_loss: 0.7738 - attrition_output_accuracy: 0.8513 - department_output_accuracy: 0.6640

In [15]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test_scaled, {'department_output': y_test__department_enc, 'attrition_output': y_test__attrition_enc})
test_results

12/12 [==============================] - 0s 886us/step - loss: 2.7363 - attrition_output_loss: 0.8011 - department_output_loss: 1.9352 - attrition_output_accuracy: 0.8098 - department_output_accuracy: 0.5190


[2.736300468444824,
 0.8011118173599243,
 1.9351886510849,
 0.8097826242446899,
 0.51902174949646]

In [16]:
# Print the accuracy for both department and attrition
print(f"Attrition predictions accuracy: {test_results[3]}")
print(f"Department predictions accuracy: {test_results[4]}")

Attrition predictions accuracy: 0.8097826242446899
Department predictions accuracy: 0.51902174949646


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. 
Accuracy is a good metric, but it might not be the best for this dataset. Since there are probably more employees staying than leaving, the model could just predict "no attrition" most of the time and still get a high accuracy. But that wouldn’t actually help in finding employees who are likely to leave.

2.
 For the Department prediction, I used Softmax because there are multiple departments. Softmax makes sure the model picks the most likely department by giving a probability for each one.

For Attrition, I used Sigmoid since there are only two choices (Yes or No). Sigmoid gives a probability between 0 and 1, which helps predict how likely an employee is to leave.

3. 
Add more features like job role, salary, or satisfaction levels to make better predictions.  
Use better metrics instead of just accuracy, check precision, recall to get a clearer idea of how well the model is performing.  
Fix class imbalance using SMOTE or class weighting to help the model detect employees who might leave.
